Brug *word2vec_tol* conda env til dette script.

*terms.csv* skal være en csv med en kolonne af ord med navnet *terms*. Output af scriptet er den samme csv-fil med en kolonne kaldet *similar* med kommasepererede ord, der minder om ordene i *terms*-kolonnen.

In [46]:
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm

tqdm.pandas()

df = pd.read_csv('data.csv')

df.dropna(inplace=True)

df.head(10)

,terms
0,rammemål/værdi
1,accept
2,Kantine
3,kompromis
4,forcus
5,hals
6,utal
7,tiltag
8,onlin
9,land


In [47]:
from dasem.gutenberg import Word2Vec

In [48]:
w2v = Word2Vec()

/home/bertil/anaconda3/envs/word2vec_tool/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [49]:
# Lav en en ny df med alle ordkombinationer

terms = pd.DataFrame(list(itertools.combinations(df['terms'], 2)))

terms.head()

,0,1
0,rammemål/værdi,accept
1,rammemål/værdi,Kantine
2,rammemål/værdi,kompromis
3,rammemål/værdi,forcus
4,rammemål/værdi,hals


# W2V similarity

In [50]:
def w2v_similarity(row):
    """
    Test w2v-lighed mellem to ord
    """
    try: 
        return w2v.similarity(row[0], row[1])
    except:
        return np.nan

terms['w2v'] = terms.progress_apply(w2v_similarity, axis=1)

terms.sort_values(by='w2v', ascending=False).head(15)

100%|██████████| 547581/547581 [00:29<00:00, 18341.53it/s]


,0,1,w2v
16809,kammerat,kollega,0.908238
147570,hjerne,nærhed,0.902608
144524,tilstedeværelse,adfærd,0.895739
147913,hjerne,fantasi,0.891704
424513,fantasi,puls,0.890048
57897,udvikling,virksomhed,0.889248
359910,kontrol,inspiration,0.888490
439512,anerkendelse,påklædning,0.886555
88388,hjælpemiddel,arrangement,0.884631
446085,oplevelse,kineser,0.881809


In [51]:
terms.sort_values(by='w2v', ascending=False).head(15)

,0,1,w2v
16809,kammerat,kollega,0.908238
147570,hjerne,nærhed,0.902608
144524,tilstedeværelse,adfærd,0.895739
147913,hjerne,fantasi,0.891704
424513,fantasi,puls,0.890048
57897,udvikling,virksomhed,0.889248
359910,kontrol,inspiration,0.888490
439512,anerkendelse,påklædning,0.886555
88388,hjælpemiddel,arrangement,0.884631
446085,oplevelse,kineser,0.881809


Lav en df med de originale ord som indeks, hvis de er over en vis værdi (CUTOFF) i w2v-lighed.
Kolonne-1 er de ord, der har lighed over CUTOFF med ordene i indeks.

In [52]:
W2V_CUTOFF = 0.85

merge = pd.DataFrame(terms[terms['w2v'] > W2V_CUTOFF].groupby(0)[1].apply(lambda x: ", ".join(x)))

merge.head(15)

,1
0,
adfærd,uvidenhed
anerkendelse,påklædning
arrangement,"eksperiment, system"
baggrund,skærm
barns,"laboratorium, redskab"
begavelse,oplevelse
bås,hylde
dannelse,læsning
egoisme,inspiration


Merge-dataframen har kun de ord, der overstiger CUTOFF. Derfor skal vi merge den tilbage på den oprindelige dataframe.

In [53]:
df = df.merge(right=merge, left_on='terms', right_index=True, how='left')

df.head(15)

,terms,1
0,rammemål/værdi,NaN
1,accept,NaN
2,Kantine,NaN
3,kompromis,NaN
4,forcus,NaN
5,hals,NaN
6,utal,"pres, håndarbejde"
7,tiltag,NaN
8,onlin,NaN
9,land,NaN


# Levenshtein distance

In [54]:
from Levenshtein import distance

def lev_similarity(row):
    """
    Test w2v-lighed mellem to ord
    """
    try: 
        return distance(row[0], row[1])
    except:
        return np.nan

terms['lev'] = terms.progress_apply(lev_similarity, axis=1)

terms.sort_values(by='lev').head(15)

100%|██████████| 547581/547581 [00:13<00:00, 40212.71it/s]


,0,1,w2v,lev
354505,klassetrin,klassetrinn,NaN,1
392519,sparing,sparring,NaN,1
55444,styk,styr,NaN,1
172533,elev,elevs,NaN,1
229328,klasse,kasse,0.577458,1
49800,Frihed,frihed,NaN,1
289051,Start,start,NaN,1
244038,mode,kode,NaN,1
20197,vane,vare,0.017064,1
263353,kø,dø,NaN,1


In [55]:
LEV_CUTOFF = 2

merge = pd.DataFrame(terms[terms['lev'] < LEV_CUTOFF].groupby(0)[1].apply(lambda x: ", ".join(x)))

merge.head(15)

,1
0,
-arbejde,arbejde
-ting,ting
Eleven,eleven
Etik,"etik, stik"
Fokus,fokus
Frihed,frihed
Ipad,ipad
Klasse,klasse
Klima,klima


In [56]:
df = df.merge(right=merge, left_on='terms', right_index=True, how='left')

df.columns = ['terms', 'w2v_words', 'lev_words']

df.head(15)

,terms,w2v_words,lev_words
0,rammemål/værdi,NaN,NaN
1,accept,NaN,NaN
2,Kantine,NaN,NaN
3,kompromis,NaN,NaN
4,forcus,NaN,focus
5,hals,NaN,hal
6,utal,"pres, håndarbejde",NaN
7,tiltag,NaN,NaN
8,onlin,NaN,NaN
9,land,NaN,"uland, band"


In [57]:
df.to_csv('output.csv')